In [7]:
def parse_file(file_path, f_id):
    with open(file_path, "r") as file:
        lines = file.readlines()

    atoms = {}
    obstacles = {}
    win_condition_id = None
    interaction_pairs = {}
    level_mapping = {}
    rules = {}

    in_movable = in_obstacle = in_interaction_set = in_level_mapping = False

    for line in lines:
        line = line.strip()

        if "movable >" in line:
            in_movable = True
            continue
        elif "obstacle >" in line:
            in_obstacle = True
            in_movable = False
            continue
        elif line.startswith("InteractionSet"):
            in_interaction_set = True
            in_obstacle = False
        elif line.startswith("LevelMapping"):
            in_level_mapping = True
            in_interaction_set = False
        elif line.startswith("TerminationSet"):
            in_level_mapping = False

        if in_movable and "> Passive" in line:
            id = line.split()[0]
            atoms[id] = None
        elif in_obstacle and "> Immovable" in line:
            id = line.split()[0]
            obstacles[id] = None
        elif in_interaction_set and "> createSprite stype=" in line:
            parts = line.split(" > createSprite stype=")
            ids1, _ = parts
            id1, id2 = ids1.split(" ")
            left_side = (int(id1), int(id2))
            right_side = parts[1].strip('"')
            interaction_pairs[right_side] = left_side
            rules[left_side] = int(right_side)
        elif in_interaction_set and "> killBoth" in line:
            parts = line.split(" > killBoth")
            ids1, _ = parts
            id1, id2 = ids1.split(" ")
            left_side = (int(id1), int(id2))
            right_side = 99999
            rules[left_side] = right_side
        elif in_level_mapping and ">" in line:
            symbol, mapping = line.split(" > ")
            id = mapping.split()[-1]
            level_mapping[id] = symbol
        elif "MultiSpriteCounter stype=" in line:
            win_condition_id = line.split('"')[1]

    # Update atoms and obstacles with symbols from level mapping
    for id in atoms.keys():
        atoms[id] = level_mapping.get(id)
    for id in obstacles.keys():
        obstacles[id] = level_mapping.get(id)

    # Find the win condition based on the win_condition_id
    win_condition = interaction_pairs.get(win_condition_id, None)

    return {
        f_id: [
            {
                "walls": None,
                "atoms": atoms,
                "obstacles": obstacles,
                "agent": (None, None),
                "win_condition": {win_condition: int(win_condition_id)},
            },
            {"rules": rules},
        ]
    }


# Example usage
file_path = "generated/generated_definition.txt"
parsed_data = parse_file(file_path, 0)
print(parsed_data)

{0: [{'walls': None, 'atoms': {'0': 'α', '3': 'δ', '11': 'μ', '13': 'ξ', '15': 'π', '17': 'ς', '18': 'σ', '10003': 'φ', '20': 'υ', '10020': 'ϭ', '10045': 'Ϲ', '10047': 'Ϻ', '10048': 'ϻ', '100032': 'я', '100036': 'ё', '10070': 'Ѕ', '10072': 'І', '10074': 'Ї', '10075': 'Ј'}, 'obstacles': {'101032': 'Ґ', '11020': 'д', '101036': 'Ғ'}, 'agent': (None, None), 'win_condition': {(11, 10048): 100032}}, {'rules': {(13, 10003): 10048, (11, 10048): 100032, (18, 20): 10003, (10045, 18): 10048, (13, 20): 10045, (10047, 20): 10048, (13, 18): 10047, (17, 20): 10020, (3, 15): 10070, (0, 10070): 10075, (0, 15): 10072, (10072, 3): 10075, (0, 3): 10074, (10074, 15): 10075, (11, 10075): 100036, (100032, 101032): 99999, (10020, 11020): 99999, (100036, 101036): 99999}}]}


In [8]:
def update_dictionary_with_layout(file_path, dictionary, f_id):
    with open(file_path, "r") as file:
        layout = file.read()

    walls = []
    agent = None
    atoms = {}
    obstacles = {}

    layout_lines = layout.split("\n")
    for y, line in enumerate(layout_lines):
        for x, symbol in enumerate(line):
            if symbol == "w":
                walls.append((x, y))
            elif symbol == "A":
                agent = (x, y)
            else:
                for key, value in dictionary[f_id][0]["atoms"].items():
                    if value == symbol:
                        key = int(key)
                        atoms[(x, y)] = key
                for key, value in dictionary[f_id][0]["obstacles"].items():
                    if value == symbol:
                        key = int(key)
                        obstacles[(x, y)] = key
            size = (x+1, y+1)

    if atoms and obstacles:
        dictionary[f_id][0]["size"] = size
        dictionary[f_id][0]["walls"] = walls
        dictionary[f_id][0]["agent"] = agent
        dictionary[f_id][0]["atoms"] = atoms
        dictionary[f_id][0]["obstacles"] = obstacles

    return dictionary


In [9]:
def parse_files(def_file_path, layout_file_paths):
    dictionary = {}
    for i, layout_file_path in enumerate(layout_file_paths):
        if i not in dictionary:
            dictionary.update(parse_file(def_file_path, i))
        dictionary = update_dictionary_with_layout(layout_file_path, dictionary, i)
    return dictionary

In [10]:
import os

def_file_path = 'generated/generated_definition.txt'  # replace with your definition file path
layout_folder_path = 'layouts'  # replace with your layouts folder path

# Get a list of all layout file paths in the layouts folder
layout_file_paths = [os.path.join(layout_folder_path, f) for f in os.listdir(layout_folder_path) if os.path.isfile(os.path.join(layout_folder_path, f))]

# Use the parse_files function
dictionary = parse_files(def_file_path, layout_file_paths)

# Now dictionary contains the parsed data for your definition file and all layout files


In [11]:
dictionary[0]

[{'walls': [(0, 0),
   (1, 0),
   (2, 0),
   (3, 0),
   (4, 0),
   (5, 0),
   (6, 0),
   (7, 0),
   (8, 0),
   (9, 0),
   (10, 0),
   (11, 0),
   (12, 0),
   (13, 0),
   (14, 0),
   (15, 0),
   (16, 0),
   (17, 0),
   (18, 0),
   (19, 0),
   (20, 0),
   (21, 0),
   (22, 0),
   (23, 0),
   (24, 0),
   (25, 0),
   (0, 1),
   (4, 1),
   (25, 1),
   (0, 2),
   (4, 2),
   (11, 2),
   (12, 2),
   (13, 2),
   (14, 2),
   (15, 2),
   (16, 2),
   (17, 2),
   (18, 2),
   (19, 2),
   (20, 2),
   (21, 2),
   (25, 2),
   (0, 3),
   (4, 3),
   (11, 3),
   (12, 3),
   (13, 3),
   (14, 3),
   (15, 3),
   (16, 3),
   (17, 3),
   (25, 3),
   (0, 4),
   (1, 4),
   (2, 4),
   (4, 4),
   (11, 4),
   (12, 4),
   (13, 4),
   (14, 4),
   (15, 4),
   (16, 4),
   (17, 4),
   (19, 4),
   (20, 4),
   (21, 4),
   (22, 4),
   (23, 4),
   (24, 4),
   (25, 4),
   (0, 5),
   (4, 5),
   (11, 5),
   (12, 5),
   (13, 5),
   (14, 5),
   (15, 5),
   (16, 5),
   (17, 5),
   (19, 5),
   (20, 5),
   (21, 5),
   (22, 5),
   (2

In [12]:
import pickle
with open("dic.pkl", "wb") as f:
    pickle.dump(dictionary, f)